In [265]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import string
import numpy as np
from sklearn.cluster import KMeans

# Download NLTK resources
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

# lscs members
lscs = pd.read_csv(
    "lscs_members.csv",  # 👈 disable quoting
    on_bad_lines="warn"      # skip/warn bad rows
)

lscs.head()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
/tmp/ipython-input-1060829184.py:22: ParserWarning: Skipping line 743: expected 13 fields, saw 15

  lscs = pd.read_csv(


,id,full_name,nickname,email,telegram,position_id,committee_id,college,program,discord,interests,contact_number,fb_link
0,11825782,"Bernaldez, Leo Mikhail B.",NaN,leo_bernaldez@dlsu.edu.ph,NaN,MEM,MEM,GCOE,BS - Chemical Engineering,NaN,NaN,09052898843,https://www.facebook.com/beltran.bernaldez
1,11941820,"De Jesus, Matthew Joseph M.",NaN,matthew.dejesus@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies (CCS),BS - Computer Science Major in Network and Inf...,NaN,"Gaming, Editing",0917 618 8375,https://www.facebook.com/matti42/
2,12033391,"Palencia, Ralph Dwanne V",NaN,ralph_palencia@dlsu.edu.ph,NaN,MEM,MEM,SOE,BS Applied Industrial Economics,NaN,"Tech, games",09567349274,fb.com/dwanne.palencia
3,12038660,"Fong, Hannah Regine C.",NaN,hannah_regine_fong@dlsu.edu.ph,NaN,MEM,MEM,CCS,BSMS - Computer Science,NaN,NaN,09177012643,fb.com/fong.hannah
4,12073067,"de Ramos, Ghrazielle Rei A.",NaN,ghrazielle_deramos@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS Computer Science Major in Software Technology,NaN,"Basketball, Sketching, Coding, Volunteering",0933 862 0716,https://web.facebook.com/ghrazielle/


In [266]:
import pandas as pd

# Define the houses
houses = {
    "Red": {"House Name": "House of Jasper", "Gem/Crystal": "Jasper", "Signature Color/Look": "earthy red/brown-red"},
    "Orange": {"House Name": "House of Carnelian", "Gem/Crystal": "Carnelian", "Signature Color/Look": "fiery orange"},
    "Yellow": {"House Name": "House of Citrine", "Gem/Crystal": "Citrine", "Signature Color/Look": "golden yellow"},
    "Green": {"House Name": "House of Aventurine", "Gem/Crystal": "Aventurine", "Signature Color/Look": "shimmering green"},
    "Light Blue": {"House Name": "House of Amazonite", "Gem/Crystal": "Amazonite", "Signature Color/Look": "turquoise blue-green"},
    "Indigo/Dark Blue": {"House Name": "House of Lapis Lazuli", "Gem/Crystal": "Lapis Lazuli", "Signature Color/Look": "deep royal blue"},
    "Purple": {"House Name": "House of Amethyst", "Gem/Crystal": "Amethyst", "Signature Color/Look": "violet to purple"},
    "White": {"House Name": "House of Moonstone", "Gem/Crystal": "Moonstone", "Signature Color/Look": "milky white with rainbow sheen"},
    "Black": {"House Name": "House of Obsidian", "Gem/Crystal": "Obsidian / Onyx", "Signature Color/Look": "jet black (volcanic glass)"},
    "Pink/Brown": {"House Name": "House of Rose Quartz", "Gem/Crystal": "Serpentine", "Signature Color/Look": "pink"}
}

# Convert to DataFrame
house_df = pd.DataFrame.from_dict(houses, orient="index").reset_index()

# Rename columns
house_df.rename(columns={"index": "Color Category"}, inplace=True)

# Add numeric ID column
house_df.insert(0, "House ID", range(len(house_df)))

# Save to CSV
house_df.to_csv("houses.csv", index=False)

house_df


,House ID,Color Category,House Name,Gem/Crystal,Signature Color/Look
0,0,Red,House of Jasper,Jasper,earthy red/brown-red
1,1,Orange,House of Carnelian,Carnelian,fiery orange
2,2,Yellow,House of Citrine,Citrine,golden yellow
3,3,Green,House of Aventurine,Aventurine,shimmering green
4,4,Light Blue,House of Amazonite,Amazonite,turquoise blue-green
5,5,Indigo/Dark Blue,House of Lapis Lazuli,Lapis Lazuli,deep royal blue
6,6,Purple,House of Amethyst,Amethyst,violet to purple
7,7,White,House of Moonstone,Moonstone,milky white with rainbow sheen
8,8,Black,House of Obsidian,Obsidian / Onyx,jet black (volcanic glass)
9,9,Pink/Brown,House of Rose Quartz,Serpentine,pink


In [267]:
specific = lscs[lscs["full_name"].str.contains("Rohann", case=False)]
specific

,id,full_name,nickname,email,telegram,position_id,committee_id,college,program,discord,interests,contact_number,fb_link
254,12343765,"Dizon, Rohann Gabriel D.",NaN,rohann_dizon@dlsu.edu.ph,NaN,VP,RND,College of Computer Studies,BS - CS ST,NaN,Volleyball,0967 087 1244,https://www.facebook.com/roedzn


In [268]:
# Clean your "interests" column
lscs["cleaned_interests"] = (
    lscs["interests"]
    .astype(str)
    .replace(r"[&/;\n|*+-]", ",", regex=True)  # added + and -
    .str.replace(r",+", ",", regex=True)        # collapse multiple commas
    .str.strip(", ")                            # strip leading/trailing commas/spaces
)

initial = lscs['cleaned_interests'].sort_values()
initial.to_csv("initial_keywords.csv")
get_nan = lscs[lscs['cleaned_interests'].str.contains('nan',case=False)]
print("empty: ", initial.shape[0])

empty:  853


In [269]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

# Step 1: Build phrase map automatically from your DataFrame
stop_words = set(stopwords.words('english'))

# Collect all words from the DataFrame
all_words = []
for text in lscs['cleaned_interests']:
    tokens = [w.lower() for w in word_tokenize(str(text)) if w not in string.punctuation and w not in stop_words]
    all_words.extend(tokens)

# Detect top bigrams
bigram_finder = BigramCollocationFinder.from_words(all_words)
top_bigrams = bigram_finder.nbest(BigramAssocMeasures.likelihood_ratio, 20)  # top 20 bigrams

# Detect top trigrams
trigram_finder = TrigramCollocationFinder.from_words(all_words)
top_trigrams = trigram_finder.nbest(TrigramAssocMeasures.likelihood_ratio, 10)  # top 10 trigrams

# Create phrase map: join words with _ so they become single tokens
phrase_map = {" ".join(b): "_".join(b) for b in top_bigrams + top_trigrams}

# Step 2: Define keyword extraction function with nouns and verbs
def extract_keywords_auto(sentence):
    sentence_lower = str(sentence).lower()

    # Apply phrase mapping
    for phrase, replacement in phrase_map.items():
        if phrase in sentence_lower:
            sentence_lower = sentence_lower.replace(phrase, replacement)

    # Tokenize
    words = word_tokenize(sentence_lower)
    # Remove punctuation and stopwords
    words = [w for w in words if w not in string.punctuation and w not in stop_words]

    # POS tagging
    tagged = pos_tag(words)

    # Keep nouns and verbs
    keywords = [word for word, tag in tagged if tag.startswith('NN') or tag.startswith('VB')]

    # Return unique keywords
    return list(set(keywords))

# Step 3: Apply to DataFrame
lscs["keywords_list"] = lscs["cleaned_interests"].apply(extract_keywords_auto)

# Save and inspect
keywords = lscs['keywords_list'].sort_values()
keywords.to_csv('keywords.csv', index=False)

# Count empty keyword rows
empty_keywords = lscs['keywords_list'].apply(lambda x: len(x) == 0).sum()
print("Number of empty keyword rows: ", empty_keywords)

# Get rows with empty keywords
empty_rows = lscs[lscs['keywords_list'].apply(lambda x: len(x) == 0)]

# Display their names (or any column you want)
empty_names = empty_rows['full_name'].tolist()
print("Empty keywords found for these names:")
print(empty_names)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Number of empty keyword rows:  2
Empty keywords found for these names:
['Rejano, Hans Martin F.', 'Arous, Adam']


In [270]:
# Convert list of keywords into a comma-separated string
lscs["keywords"] = lscs["keywords_list"].apply(lambda x: ", ".join(x) if x else "")

# Sort the keywords column alphabetically
lscs_sorted = lscs.sort_values(by="keywords")

# Optional: inspect
print(lscs_sorted[['keywords']].head())

# Save the sorted CSV
lscs_sorted[['keywords']].to_csv("keywords_sorted.csv", index=False)

# If you want to save the original list column too
lscs_sorted['keywords_list'].to_csv("keywords_list_sorted.csv", index=False)

lscs_sorted

                                              keywords
642                                                   
5                                                     
70                            ai, shounen, programming
27                animanga, science, tech, video_games
793  anime, basketball, language, gaming, learning,...


,id,full_name,nickname,email,telegram,position_id,committee_id,college,program,discord,interests,contact_number,fb_link,cleaned_interests,keywords_list,keywords
642,12505196,"Arous, Adam",NaN,adam_arous@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Information Technology,NaN,many,0976 216 3937,https://www.facebook.com/adam.arousii?mibextid...,many,[],
5,12107884,"Rejano, Hans Martin F.",NaN,hans_rejano@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Computer Science,NaN,?,0929 236 6554,fb.com/rejano03,?,[],
70,12208973,"Ruiz, James Christian, T",NaN,james_christian_ruiz@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies,"BS - Computer Science, majoring in Software Te...",NaN,"Programming, Astronomy, Shounen Ai",0906 453 5445,https://www.facebook.com/jameschristian.ruiz,"Programming, Astronomy, Shounen Ai","[ai, shounen, programming]","ai, shounen, programming"
27,12205486,"De Grano, Justin Patrick M.",NaN,justin_degrano@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies,BS - Information Technology,NaN,"Science/tech, video games, animanga",09694562865,NaN,"Science,tech, video games, animanga","[animanga, science, tech, video_games]","animanga, science, tech, video_games"
793,12512478,"Sergio, Sean Gabriel Steven G.",NaN,sean_sergio@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Science,BS Computer Science Major in Software Technology,NaN,"Basketball, Art, Gaming, Game dev, Language Le...",09695320125,https://www.facebook.com/sean.sergioSS/,"Basketball, Art, Gaming, Game dev, Language Le...","[anime, basketball, language, gaming, learning...","anime, basketball, language, gaming, learning,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,12506370,"Pornasdoro, Jennifer Heather Anne R.",NaN,jennifer_pornasdoro@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies,BS Information Systems,NaN,Writing,09190018678,https://www.facebook.com/share/1ELMfGJUi7/?mib...,Writing,[writing],writing
56,12207799,"Hermoso, Mikaela M.",NaN,mikaela_hermoso@dlsu.edu.ph,NaN,AVP,PUBLI,CLA,AB- Literature,NaN,writing,09297785900,https://www.facebook.com/share/178GVFR7vW/?mib...,writing,[writing],writing
653,12505374,"Esguerra, Roland Thomas III, P.",NaN,roland_esguerra@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies,BS - Computer Science Major in Software Techno...,NaN,"Game development, 2D art, writing",+ 63 915 106 1456,https://www.facebook.com/roland.thomas.esguerra/,"Game development, 2D art, writing","[writing, game_development, art]","writing, game_development, art"
338,12410381,"De Gracia, Kaleela Ysabel B.",NaN,kaleela_degracia@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS Computer Science Major in Computer Systems ...,NaN,"Gaming, Reading and Writing",0995 745 1018,https://www.facebook.com/kaleelaysabel.degracia,"Gaming, Reading and Writing","[writing, gaming, reading]","writing, gaming, reading"


In [271]:
# Only vectorize non-empty rows
non_empty = lscs["keywords"] != ""
vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split(", "))
X = vectorizer.fit_transform(lscs.loc[non_empty, "keywords"])


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [272]:
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
labels = kmeans.fit_predict(X)

# Compute distances to cluster centers
distances = kmeans.transform(X)  # shape: (n_samples, n_clusters)

# Sort points by distance to each cluster center
sorted_idx = np.argsort(distances, axis=0)

# Assign points evenly to clusters
cluster_size = X.shape[0] // n_clusters
balanced_labels = np.zeros(X.shape[0], dtype=int)

assigned = np.zeros(X.shape[0], dtype=bool)
for cluster_id in range(n_clusters):
    count = 0
    for idx in sorted_idx[:, cluster_id]:
        if not assigned[idx] and count < cluster_size:
            balanced_labels[idx] = cluster_id
            assigned[idx] = True
            count += 1

# Assign remaining points (if X.shape[0] % n_clusters != 0)
for i, flag in enumerate(assigned):
    if not flag:
        balanced_labels[i] = np.random.randint(0, n_clusters)

# Save balanced labels into your DataFrame
lscs.loc[non_empty, "assigned_house"] = balanced_labels

# For rows outside `non_empty`, assign random clusters
lscs.loc[~non_empty, "assigned_house"] = np.random.randint(0, n_clusters, size=(~non_empty).sum())

# Ensure integer type
lscs["assigned_house"] = lscs["assigned_house"].astype(int)

lscs.to_csv('assigned_houses.csv', index=False)

In [273]:
lscs['assigned_house'].value_counts().sort_index()


,count
assigned_house,
0,85
1,86
2,85
3,86
4,85
5,86
6,85
7,85
8,85


In [274]:
# Create a mapping from House ID to House Name
house_id_to_name = dict(zip(house_df["House ID"], house_df["House Name"]))

# Replace assigned_house numbers with house names
lscs["assigned_house"] = lscs["assigned_house"].map(house_id_to_name)

# Save to CSV if needed
lscs.to_csv("assigned_houses.csv", index=False)

lscs.head()

,id,full_name,nickname,email,telegram,position_id,committee_id,college,program,discord,interests,contact_number,fb_link,cleaned_interests,keywords_list,keywords,assigned_house
0,11825782,"Bernaldez, Leo Mikhail B.",NaN,leo_bernaldez@dlsu.edu.ph,NaN,MEM,MEM,GCOE,BS - Chemical Engineering,NaN,NaN,09052898843,https://www.facebook.com/beltran.bernaldez,nan,[nan],nan,House of Obsidian
1,11941820,"De Jesus, Matthew Joseph M.",NaN,matthew.dejesus@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies (CCS),BS - Computer Science Major in Network and Inf...,NaN,"Gaming, Editing",0917 618 8375,https://www.facebook.com/matti42/,"Gaming, Editing","[editing, gaming]","editing, gaming",House of Aventurine
2,12033391,"Palencia, Ralph Dwanne V",NaN,ralph_palencia@dlsu.edu.ph,NaN,MEM,MEM,SOE,BS Applied Industrial Economics,NaN,"Tech, games",09567349274,fb.com/dwanne.palencia,"Tech, games","[games, tech]","games, tech",House of Amazonite
3,12038660,"Fong, Hannah Regine C.",NaN,hannah_regine_fong@dlsu.edu.ph,NaN,MEM,MEM,CCS,BSMS - Computer Science,NaN,NaN,09177012643,fb.com/fong.hannah,nan,[nan],nan,House of Moonstone
4,12073067,"de Ramos, Ghrazielle Rei A.",NaN,ghrazielle_deramos@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS Computer Science Major in Software Technology,NaN,"Basketball, Sketching, Coding, Volunteering",0933 862 0716,https://web.facebook.com/ghrazielle/,"Basketball, Sketching, Coding, Volunteering","[volunteering, basketball, coding, sketching]","volunteering, basketball, coding, sketching",House of Amethyst


In [275]:
# Create a clean copy
official_lscs = lscs.copy()

# Drop unwanted columns
columns_to_drop = ["cleaned_interests", "keywords_list", "keywords"]
official_lscs = official_lscs.drop(columns=[col for col in columns_to_drop if col in official_lscs.columns])

# Save to CSV if needed
official_lscs.to_csv("official_lscs.csv", index=False)

official_lscs.head()

,id,full_name,nickname,email,telegram,position_id,committee_id,college,program,discord,interests,contact_number,fb_link,assigned_house
0,11825782,"Bernaldez, Leo Mikhail B.",NaN,leo_bernaldez@dlsu.edu.ph,NaN,MEM,MEM,GCOE,BS - Chemical Engineering,NaN,NaN,09052898843,https://www.facebook.com/beltran.bernaldez,House of Obsidian
1,11941820,"De Jesus, Matthew Joseph M.",NaN,matthew.dejesus@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies (CCS),BS - Computer Science Major in Network and Inf...,NaN,"Gaming, Editing",0917 618 8375,https://www.facebook.com/matti42/,House of Aventurine
2,12033391,"Palencia, Ralph Dwanne V",NaN,ralph_palencia@dlsu.edu.ph,NaN,MEM,MEM,SOE,BS Applied Industrial Economics,NaN,"Tech, games",09567349274,fb.com/dwanne.palencia,House of Amazonite
3,12038660,"Fong, Hannah Regine C.",NaN,hannah_regine_fong@dlsu.edu.ph,NaN,MEM,MEM,CCS,BSMS - Computer Science,NaN,NaN,09177012643,fb.com/fong.hannah,House of Moonstone
4,12073067,"de Ramos, Ghrazielle Rei A.",NaN,ghrazielle_deramos@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS Computer Science Major in Software Technology,NaN,"Basketball, Sketching, Coding, Volunteering",0933 862 0716,https://web.facebook.com/ghrazielle/,House of Amethyst


In [276]:
top_n = 10  # number of top keywords per cluster
cluster_data = []

for cluster_id in lscs['cluster'].unique():
    cluster_rows = lscs[lscs['cluster'] == cluster_id]
    all_keywords = sum(cluster_rows['keywords_list'], [])  # flatten lists
    keyword_counts = Counter(all_keywords)
    top_keywords = [kw for kw, _ in keyword_counts.most_common(top_n)]

    cluster_data.append({
        'Cluster': cluster_id,
        'Count': len(cluster_rows),
        **{f'Top{i+1}': kw for i, kw in enumerate(top_keywords)}
    })

# Create DataFrame
df_cluster_summary = pd.DataFrame(cluster_data)
df_cluster_sort = df_cluster_summary.sort_values('Cluster')

# Display
print(df_cluster_sort)


KeyError: 'cluster'